In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('cleveland297.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [5]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [6]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [7]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [8]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [9]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 3s 225ms/step - loss: 0.7654 - accuracy: 0.5393 - val_loss: 0.6742 - val_accuracy: 0.6555
Epoch 2/100
4/4 [==============================] - 0s 58ms/step - loss: 0.5898 - accuracy: 0.6854 - val_loss: 0.6579 - val_accuracy: 0.6639
Epoch 3/100
4/4 [==============================] - 0s 54ms/step - loss: 0.5510 - accuracy: 0.7247 - val_loss: 0.6477 - val_accuracy: 0.6471
Epoch 4/100
4/4 [==============================] - 0s 61ms/step - loss: 0.4834 - accuracy: 0.7753 - val_loss: 0.6453 - val_accuracy: 0.6134
Epoch 5/100
4/4 [==============================] - 0s 52ms/step - loss: 0.5757 - accuracy: 0.7472 - val_loss: 0.6426 - val_accuracy: 0.5966
Epoch 6/100
4/4 [==============================] - 0s 26ms/step - loss: 0.4423 - accuracy: 0.7528 - val_loss: 0.6435 - val_accuracy: 0.5966
Epoch 7/100
4/4 [==============================] - 0s 26ms/step - loss: 0.4440 - accuracy: 0.7921 - val_loss: 0.6429 - val_accuracy: 0.5798
Epoch 8/100
4/4 [==

4/4 [==============================] - 0s 23ms/step - loss: 0.0716 - accuracy: 0.9888 - val_loss: 0.5130 - val_accuracy: 0.7563
Epoch 60/100
4/4 [==============================] - 0s 24ms/step - loss: 0.0922 - accuracy: 0.9775 - val_loss: 0.5125 - val_accuracy: 0.7563
Epoch 61/100
4/4 [==============================] - 0s 27ms/step - loss: 0.0916 - accuracy: 0.9551 - val_loss: 0.5110 - val_accuracy: 0.7647
Epoch 62/100
4/4 [==============================] - 0s 23ms/step - loss: 0.1136 - accuracy: 0.9382 - val_loss: 0.5186 - val_accuracy: 0.7815
Epoch 63/100
4/4 [==============================] - 0s 26ms/step - loss: 0.0725 - accuracy: 0.9775 - val_loss: 0.5165 - val_accuracy: 0.7731
Epoch 64/100
4/4 [==============================] - 0s 26ms/step - loss: 0.1013 - accuracy: 0.9607 - val_loss: 0.5172 - val_accuracy: 0.7647
Epoch 65/100
4/4 [==============================] - 0s 21ms/step - loss: 0.0537 - accuracy: 0.9944 - val_loss: 0.5203 - val_accuracy: 0.7647
Epoch 66/100
4/4 [========

In [10]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


4/4 [==============================] - 0s 3ms/step - loss: 0.5104 - accuracy: 0.7479
Test Loss: 0.5104
Test Accuracy: 0.7479


In [11]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


4/4 [==============================] - 0s 3ms/step
Confusion Matrix:
[[56  4]
 [26 33]]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.93      0.79        60
           1       0.89      0.56      0.69        59

    accuracy                           0.75       119
   macro avg       0.79      0.75      0.74       119
weighted avg       0.79      0.75      0.74       119



In [12]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [13]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [14]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 16s 1s/step - loss: 0.6821 - accuracy: 0.6461 - val_loss: 0.6890 - val_accuracy: 0.7143
Epoch 2/100
4/4 [==============================] - 0s 98ms/step - loss: 0.6448 - accuracy: 0.7247 - val_loss: 0.6812 - val_accuracy: 0.6639
Epoch 3/100
4/4 [==============================] - 0s 103ms/step - loss: 0.6044 - accuracy: 0.7191 - val_loss: 0.6664 - val_accuracy: 0.6471
Epoch 4/100
4/4 [==============================] - 0s 100ms/step - loss: 0.5719 - accuracy: 0.7079 - val_loss: 0.6524 - val_accuracy: 0.6134
Epoch 5/100
4/4 [==============================] - 0s 113ms/step - loss: 0.5638 - accuracy: 0.7360 - val_loss: 0.6413 - val_accuracy: 0.6134
Epoch 6/100
4/4 [==============================] - 0s 105ms/step - loss: 0.5641 - accuracy: 0.7191 - val_loss: 0.6390 - val_accuracy: 0.6303
Epoch 7/100
4/4 [==============================] - 0s 60ms/step - loss: 0.5489 - accuracy: 0.7303 - val_loss: 0.6390 - val_accuracy: 0.6471
Epoch 8/100
4/4 [

In [15]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


4/4 [==============================] - 0s 11ms/step - loss: 0.6193 - accuracy: 0.6723
Test Loss: 0.6193
Test Accuracy: 0.6723


In [16]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


4/4 [==============================] - 3s 11ms/step
Confusion Matrix:
[[56  4]
 [35 24]]

Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.93      0.74        60
           1       0.86      0.41      0.55        59

    accuracy                           0.67       119
   macro avg       0.74      0.67      0.65       119
weighted avg       0.74      0.67      0.65       119



In [17]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [18]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [19]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
4/4 [==============================] - 23s 1s/step - loss: 0.8200 - accuracy: 0.5000 - val_loss: 0.6911 - val_accuracy: 0.5126
Epoch 2/100
4/4 [==============================] - 1s 179ms/step - loss: 0.5027 - accuracy: 0.7809 - val_loss: 0.6892 - val_accuracy: 0.5126
Epoch 3/100
4/4 [==============================] - 1s 211ms/step - loss: 0.5412 - accuracy: 0.7697 - val_loss: 0.6881 - val_accuracy: 0.5126
Epoch 4/100
4/4 [==============================] - 1s 170ms/step - loss: 0.5430 - accuracy: 0.7472 - val_loss: 0.6874 - val_accuracy: 0.5126
Epoch 5/100
4/4 [==============================] - 1s 186ms/step - loss: 0.5320 - accuracy: 0.7809 - val_loss: 0.6870 - val_accuracy: 0.5126
Epoch 6/100
4/4 [==============================] - 0s 99ms/step - loss: 0.4672 - accuracy: 0.7640 - val_loss: 0.6878 - val_accuracy: 0.5042
Epoch 7/100
4/4 [==============================] - 0s 106ms/step - loss: 0.4319 - accuracy: 0.8146 - val_loss: 0.6881 - val_accuracy: 0.5042
Epoch 8/100
4/4 

Epoch 59/100
4/4 [==============================] - 0s 107ms/step - loss: 0.1667 - accuracy: 0.9270 - val_loss: 0.4774 - val_accuracy: 0.7395
Epoch 60/100
4/4 [==============================] - 1s 145ms/step - loss: 0.1548 - accuracy: 0.9438 - val_loss: 0.4750 - val_accuracy: 0.7647


In [20]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


4/4 [==============================] - 0s 18ms/step - loss: 0.4411 - accuracy: 0.8067
Test Loss: 0.4411
Test Accuracy: 0.8067


In [21]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


4/4 [==============================] - 5s 20ms/step
Confusion Matrix:
[[45 15]
 [ 8 51]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.75      0.80        60
           1       0.77      0.86      0.82        59

    accuracy                           0.81       119
   macro avg       0.81      0.81      0.81       119
weighted avg       0.81      0.81      0.81       119



In [22]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



4/4 [==============================] - 0s 18ms/step


In [23]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [24]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.7647


In [25]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[57  3]
 [25 34]]
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.95      0.80        60
           1       0.92      0.58      0.71        59

    accuracy                           0.76       119
   macro avg       0.81      0.76      0.76       119
weighted avg       0.81      0.76      0.76       119

Ensemble Accuracy: 0.7647


In [26]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [27]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [28]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [29]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [30]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [31]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8319


In [32]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[56  4]
 [16 43]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.93      0.85        60
           1       0.91      0.73      0.81        59

    accuracy                           0.83       119
   macro avg       0.85      0.83      0.83       119
weighted avg       0.85      0.83      0.83       119

Ensemble Accuracy: 0.8319


In [33]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [34]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8151
Confusion Matrix:
[[56  4]
 [18 41]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.93      0.84        60
           1       0.91      0.69      0.79        59

    accuracy                           0.82       119
   macro avg       0.83      0.81      0.81       119
weighted avg       0.83      0.82      0.81       119

Ensemble Accuracy: 0.8151
